In [1]:
#!pip install n2v
#!pip install albumentations

In [1]:
# We import all our dependencies.
from n2v.models import N2VConfig, N2V
import numpy as np
from csbdeep.utils import plot_history
from n2v.utils.n2v_utils import manipulate_val_data
from n2v.internals.N2V_DataGenerator import N2V_DataGenerator
from matplotlib import pyplot as plt
import urllib
import os
import glob
import zipfile
import cv2
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
import albumentations as albu
import json
from functools import partial

import ssl
ssl._create_default_https_context = ssl._create_unverified_context


2023-05-02 19:30:20.766993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
INBREAST_PATH="/kaggle/input/mias-cbis-ddsm-inbreast/Mammographies/INBreast"
MIAS_PATH="/kaggle/input/mias-cbis-ddsm-inbreast/Mammographies/MIAS"
CBIS_DDSM_PATH="/kaggle/input/mias-cbis-ddsm-inbreast/Mammographies/CBIS-DDSM"

In [3]:
input_shape=(1600,1600)

In [4]:
# a name used to identify the model
model_name = 'n2v_2D'
# the base directory in which our model will live
basedir = '/kaggle/working/models'

# We are now creating our network model.
model = N2V(None, model_name, basedir=basedir)

2023-05-02 19:30:25.535738: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-02 19:30:25.543942: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-02 19:30:25.544146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Loading network weights from 'weights_best.h5'.


In [5]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = np.array(image).astype("uint8")
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)



In [6]:
from tensorflow.keras.utils import load_img

def load_img_from_path_no_resize(file_path):
    img = tf.io.read_file(file_path)
    img = tf.io.decode_image(img, channels=1)
    return img

def resize_correct_side(image):
    h_in = image.shape[0]
    w_in = image.shape[1]
    
    input_is_wider = h_in <= w_in
    output_is_wider = input_shape[0] <= input_shape[1]
    
    increase_width_first = abs(h_in/input_shape[0] - 1.0) < abs(w_in/input_shape[1] - 1.0)
    N = input_shape[1] if increase_width_first else input_shape[0]
    if input_is_wider != output_is_wider:
        return albu.LongestMaxSize(max_size=N)
    else:
        return albu.SmallestMaxSize(max_size=N)
    

def apply_albumentations(image, bboxes):
    image = image.numpy().astype("uint8")    
    
    transform = albu.Compose([
        resize_correct_side(image),
        albu.PadIfNeeded(min_height=input_shape[0], min_width=input_shape[1], border_mode=0, value=(0,0,0)),
        albu.CLAHE(clip_limit=(1,10),p=1),
        #albu.Rotate(p=1, border_mode=cv2.BORDER_CONSTANT,)
        ],
        bbox_params=albu.BboxParams(format='coco', label_fields=["abnormality"])
    )
    
    transform_data = transform(image = image, bboxes=bboxes, abnormality=[1]*len(bboxes))
    new_img = transform_data["image"]
    new_boxes = transform_data["bboxes"]
    return new_img.astype("uint8"), new_boxes

def preprocess_img_and_boxes(image, boxes):
    image, boxes = apply_albumentations(image,boxes)
    return image, boxes

In [7]:
with open(os.path.join(INBREAST_PATH,"roi_images.json"),"r") as f:
    rois_inbreast = json.load(f)
    
with open(os.path.join(CBIS_DDSM_PATH,"roi_images.json"),"r") as f:
    rois_cbis_ddsm = json.load(f)
    
with open(os.path.join(MIAS_PATH,"roi_images.json"),"r") as f:
    rois_mias = json.load(f)

print(len(rois_inbreast), len(rois_cbis_ddsm), len(rois_mias))

343 3103 111


In [8]:
from collections import defaultdict

datasets_rois = {"mias": rois_mias, "inbreast": rois_inbreast, "cbis_ddsm": rois_cbis_ddsm}

dataset_to_dir = {"mias": MIAS_PATH,"inbreast": INBREAST_PATH,"cbis_ddsm": CBIS_DDSM_PATH}

##Check how balanced are abnormalities
types_ab = defaultdict(int)
for dataset, rois in datasets_rois.items():
    for k, rois in rois.items():
        for roi in rois:
            types_ab[roi["type"]] += 1
types_ab

defaultdict(int,
            {'CIRC': 24,
             'MISC': 15,
             'ASYM': 15,
             'ARCH': 19,
             'SPIC': 19,
             'CALC': 27,
             'Calcification': 7142,
             'Spiculated Region': 14,
             'Mass': 1812,
             'Cluster': 27,
             'Assymetry': 1,
             'Asymmetry': 5,
             'Distortion': 3,
             'Calcifications': 1,
             'Unnamed': 2,
             'Point 3': 1,
             'Point 1': 1,
             'Spiculated region': 1,
             '': 1,
             'Espiculated Region': 1,
             'Calc': 1872})

In [9]:
name_2_name= {'CIRC': "Mass",
              'MISC': "Unknown",
              'ASYM': "Assymetry",
              'ARCH': "Distortion",
              'SPIC': "Spiculated Region",
              'CALC': "Calcification",
              'Calcification': "Calcification",
              'Spiculated Region': "Spiculated Region",
              'Mass': "Mass",
              'Cluster': "Cluster",
              'Assymetry': "Assymetry",
              'Asymmetry': "Assymetry",
              'Distortion': "Distortion",
              'Calcifications': "Calcification",
              'Unnamed': "Unknown",
              'Point 3': "Unknown",
              'Point 1': "Unknown",
              'Spiculated region': "Spiculated Region",
              '': "Unknown",
              'Calc': "Calcification",
              'Espiculated Region': "Spiculated Region"}

In [10]:
def generate_yolo_box(bboxes, image_shape):
    bboxes = np.array(bboxes)
    #Calc the xcenter and ycenter. bboxes = [xmin, ymin, w, d]
    xcenter = bboxes[0] + bboxes[2]/2
    ycenter = bboxes[1] + bboxes[3]/2
    
    #Calc proportion to image
    xcenter = xcenter / image_shape[1]
    ycenter = ycenter / image_shape[0]
    w = bboxes[2] / image_shape[1]
    h = bboxes[3] / image_shape[0]
    return xcenter, ycenter, w, h   

def check_type_abnormality(ab):
    return name_2_name[ab["type"]] in ["Calcification", "Mass"]

# We need to check the space in the box since some small abnormalities were set with a size of 5 pixels
# and can be outside the image area.
def create_and_check_box(box, image_shape):
    diff_x = box["x"] + box["w"] - image_shape[1]
    diff_y = box["y"] + box["h"] - image_shape[0]
    
    if diff_x > 0:
        box["x"] = image_shape[1] - box["w"]
    if diff_y > 0:
        box["y"] = image_shape[0] - box["h"]
    if box["x"] < 0:
        x = 0
    if box["y"] < 0:
        y = 0
        
    return [box["x"], box["y"], box["w"], box["h"]]

In [25]:
# generate YOLO data
YOLO_BASE_DIR="/kaggle/working/yolo_data"
def get_yolo_dir(type_set, label_or_image):
    assert type_set in ["train", "val", "test"]
    assert label_or_image in ["labels", "images"]
    return os.path.join(YOLO_BASE_DIR, type_set, label_or_image)

for type_set in ["train", "val", "test"]:
    for type_data in ["labels","images"]:
        os.makedirs(get_yolo_dir(type_set, type_data), exist_ok=True)

In [26]:
for dataset, rois in datasets_rois.items():
    for file, roi in rois.items():
        type_set = np.random.choice(["train", "val", "test"], p=[0.7, 0.2, 0.1])
        
        roi = list(filter(lambda r: check_type_abnormality(r), roi))
        if not roi:
            continue

        path = os.path.join(dataset_to_dir[dataset], file+".png")
        img = load_img_from_path_no_resize(path)
        
        boxes = [ create_and_check_box(box, img.shape) for box in roi ]
        
        preprocessed_img, preprocessed_boxes = preprocess_img_and_boxes(img, boxes)
        denoised_image=model.predict(np.array(preprocessed_img).reshape(input_shape), axes="YX")
        with open(os.path.join(get_yolo_dir(type_set, "labels"), file+".txt"),"w") as f:
            for box in preprocessed_boxes:
                yolo_box = generate_yolo_box(box, denoised_image.shape[:2])
                f.write(f"0 {yolo_box[0]} {yolo_box[1]} {yolo_box[2]} {yolo_box[3]} \n")

        cv2.imwrite(f"{os.path.join(get_yolo_dir(type_set, 'images'), file+'.png')}", denoised_image)

The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 34ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 26ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 

1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 17ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint

1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint

1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 34ms/step
The input image is of type uint

1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 28ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint

...

1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 22ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint

1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 25ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 29ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 34ms/step
The input image is of type uint

1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 18ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 30ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 24ms/step
The input image is of type uint

1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 21ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 25ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint

1/1 [==============================] - 0s 24ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 23ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 26ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 19ms/step
The input image is of type uint8 and will be casted to float32 for prediction.
1/1 [==============================] - 0s 20ms/step
The input image is of type uint